In [ ]:
import csv
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)

from collections import defaultdict

csv.field_size_limit(100000000)
from wordcloud import WordCloud, STOPWORDS
stopwordsi = set(STOPWORDS)

import random

random.seed(100)

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import re
from nltk.stem.porter import PorterStemmer
import nltk
from datetime import datetime
import eli5


In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import json
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy  

In [ ]:
train=pd.read_csv('train.csv')

#Dropping NA values only from text column
train = train.dropna(subset=['text'])

print(train.shape)

In [ ]:
#Converting text to lower case 
train['text'] = train['text'].apply(lambda word: " ".join(y.lower() for y in word.split()))

#removing duplicate records
train=train.drop_duplicates(subset='text', keep=False)

#Remove punctuations
train['text'] = train['text'].apply(lambda word: re.sub(r'[^\w\s.]', '', word))
train['text'] = train['text'].apply(lambda word: re.sub(r'[-]', ' ', word))
train['text'] = train['text'].apply(lambda word: re.sub(r'[_]', '', word))

#Removing any numbers and alphanumeric texts
train['text'] = train['text'].apply(lambda word: re.sub(r'[0-9]', '', word))

#Retains only English alphabetical words
train['text'] = [' '.join(w for w in p.split() if re.match(r'[a-z]', w)) for p in train['text']]

In [ ]:
#Named Entity Recognition replacement
#Takes 40 minutes to run
def tag_ne(content):
    doc = nlp(content)
    text = doc.text
    for ent in doc.ents:
        text = re.sub(ent.text, ent.label_, text)
    return text

print(datetime.now().time())
train['text']=train['text'].apply(tag_ne)
print(datetime.now().time())

In [ ]:
#removes stop words
stop_words = set(STOPWORDS)
train['text']=train['text'].apply(lambda word: " ".join(w for w in nltk.word_tokenize(word) if w not in stop_words))

In [ ]:
###Removing non-english words

###commented because it removes words having fullstop next to it, hence not able to split them by sentences,/
###as sentences require full stop after them

###Can perform this step after splitting the text into sentences

#englishwords = set(nltk.corpus.words.words())
#train['text'] = train['text'].apply(lambda word: " ".join(w for w in nltk.wordpunct_tokenize(word) if w in englishwords))

In [ ]:
#Removes words that have length less than 3

#train['text']=train['text'].apply(lambda word: " ".join(w for w in word.split() if re.match(r'\b\w{3,}\b', w)))

In [ ]:
#To perform stemming
stemmer = SnowballStemmer(language='english')
train['text'] = train['text'].apply(lambda word: " ".join([stemmer.stem(w) for w in word.split()]))

In [ ]:
#Splitting the text into reliable and fake, to distinguish between the two

reliable = train[train['label']==0]
fake = train[train['label']==1]

#counting the number of words in text column
reliable["num_words"] = reliable["text"].apply(lambda x: len(str(x).split()))

fake["num_words"] = fake["text"].apply(lambda x: len(str(x).split()))


print("fake mean number of words:", fake.num_words.mean())
print("reliable mean number of words:", reliable.num_words.mean())


In [ ]:
#word cloud for reliable 'text' column

mpl.rcParams['font.size']=12
mpl.rcParams['savefig.dpi']=100
mpl.rcParams['figure.subplot.bottom']=.1
stopwords = set(STOPWORDS)

wordcloudR = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=50,
                          max_font_size=80, 
                          random_state=42,
                         ).generate(str(reliable['text']))
 
fig = plt.figure(1)
plt.imshow(wordcloudR)

In [ ]:
#word cloud for fake 'text' column

mpl.rcParams['font.size']=12
mpl.rcParams['savefig.dpi']=100
mpl.rcParams['figure.subplot.bottom']=.1
stopwords = set(STOPWORDS)

wordcloudFake = WordCloud(
                          background_color='white',
                          stopwords=stopwords,
                          max_words=50,
                          max_font_size=80, 
                          random_state=42
                         ).generate(str(fake['text']))
 
fig = plt.figure(1)
plt.imshow(wordcloudFake)

In [ ]:
## custom function for ngram generation ##
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

## custom function for horizontal bar chart ##
def horizontal_bar_chart(df, color):
    trace = go.Bar(
        y=df["word"].values[::-1],
        x=df["wordcount"].values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        )
    )
    return trace

In [ ]:
for sent in reliable["text"]:
    for word in generate_ngrams(sent,1):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
reliable1g = horizontal_bar_chart(fd_sorted.head(50), 'orange')

freq_dict = defaultdict(int)
for sent in fake.text:
    for word in generate_ngrams(sent,1):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
fake1g = horizontal_bar_chart(fd_sorted.head(20), 'orange')

fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04,horizontal_spacing=0.15,
                          subplot_titles=["Reliable text", 
                                          "Fake text"])
fig.append_trace(reliable1g, 1, 1)
fig.append_trace(fake1g, 1, 2)
fig['layout'].update(height=500, width=1000, paper_bgcolor='rgb(233,233,233)', title="Frequent 1-gram Word Count Plots")
py.iplot(fig, filename='word-plots')

In [ ]:
#Bigrams chart
freq_dict = defaultdict(int)
for sent in reliable["text"]:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
reliable2g = horizontal_bar_chart(fd_sorted.head(20), 'orange')


freq_dict = defaultdict(int)
for sent in fake['text']:
    for word in generate_ngrams(sent,2):
        freq_dict[word] += 1
fd_sorted = pd.DataFrame(sorted(freq_dict.items(), key=lambda x: x[1])[::-1])
fd_sorted.columns = ["word", "wordcount"]
fake2g = horizontal_bar_chart(fd_sorted.head(20), 'orange')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=2, vertical_spacing=0.04,horizontal_spacing=0.15,
                          subplot_titles=["Reliable text", 
                                          "Fake text"])
fig.append_trace(reliable2g, 1, 1)
fig.append_trace(fake2g, 1, 2)
fig['layout'].update(height=500, width=1000, paper_bgcolor='rgb(233,233,233)', title="Frequent Bigram Word Count Plots")
py.iplot(fig, filename='word-plots')

In [ ]:
# Get the tfidf vectors 
tfidf_vec = TfidfVectorizer(stop_words='english')
tfidf_vec.fit_transform(train['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train['text'].values.tolist())


In [ ]:
#Displaying the word features

print(tfidf_vec.get_feature_names())

print("shape:", train_tfidf.shape)

In [ ]:
#Storing 'label' column values to train_y
train_y=train['label'].values

#Split into training and test set in 70:30 ratio
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_tfidf, train_y, random_state=0,test_size=0.3)

In [ ]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()

lsvc.fit(X_train, y_train)

print('Accuracy of LSVC classifier on training set: {0}'.format(lsvc.score(X_train, y_train)))
predictions = lsvc.predict(X_test)

print('Accuracy of LSVC classifier on test set:')
print(accuracy_score(y_test, predictions))

In [ ]:
eli5.show_weights(lsvc, vec=tfidf_vec, top=20, feature_filter=lambda x: x != '<BIAS>')

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier(max_depth=3)
dt = dt.fit(X_train, y_train)

In [ ]:
#import graphviz 
#dot_data = tree.export_graphviz(clf, out_file=None) 
#graph = graphviz.Source(dot_data) 
#graph.render("dataLatest")

In [ ]:
print('Accuracy of DT classifier on training set: {:.2f}'
     .format(dt.score(X_train, y_train)))
print('Accuracy of DT classifier on test set: {:.2f}'
     .format(dt.score(X_test, y_test)))

In [ ]:
eli5.show_weights(dt, vec=tfidf_vec, top=100, feature_filter=lambda x: x != '<BIAS>')